In [18]:
import json
import rdflib
from rdflib import Graph, RDF, XSD

In [19]:
model_name = "bert-base-cased"

In [20]:
folder_path = f'./models/{model_name}/merged_data.json'

In [22]:
with open(folder_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [23]:
# Define a custom namespace for your data
KOAI = rdflib.Namespace("http://knowledgemap.kr/koai/def/type/")

# Create a new RDF graph
g = rdflib.Graph()

# Add namespace prefix to the graph
g.bind("koai", KOAI)

In [24]:
# Define a function to add a model's information to the RDF graph dynamically
def add_model_info_to_graph(model_info):
    # Generate model URI based on the model ID
    model_uri = KOAI[model_info["id"].replace("-", "_").replace("/", "_")]
    g.add((model_uri, RDF.type, KOAI.AIModel))
    
    # Loop through each key in the model_info dictionary and add to RDF
    for key, value in model_info.items():
        if key in ['finetune', 'adapter', 'merge', 'quantized', 'dataset'] or value is None:
            continue  # Skip keys that need to be handled separately

        if isinstance(value, list):  # If the value is a list, handle it separately
            for item in value:
                item_uri = KOAI[item.replace("-", "_").replace("/", "_")] if isinstance(item, str) else rdflib.Literal(item)
                g.add((model_uri, KOAI[key], item_uri))
        elif isinstance(value, bool):  # If the value is a boolean
            g.add((model_uri, KOAI[key], rdflib.Literal(value, datatype=XSD.boolean)))
        elif isinstance(value, int):  # If the value is an integer
            g.add((model_uri, KOAI[key], rdflib.Literal(value, datatype=XSD.integer)))
        elif isinstance(value, str):  # If the value is a string
            g.add((model_uri, KOAI[key], rdflib.Literal(value)))
        else:  # Default case for other data types
            g.add((model_uri, KOAI[key], rdflib.Literal(str(value))))

    # Handle finetuned models separately
    if model_info.get("finetune"):
        for finetuned_model in model_info["finetune"]:
            finetuned_model_uri = KOAI[finetuned_model.replace("-", "_").replace("/", "_")]
            g.add((finetuned_model_uri, RDF.type, KOAI.FinetunedModel))
            g.add((finetuned_model_uri, KOAI.isFineTunedFrom, model_uri))
            g.add((model_uri, KOAI.hasFineTunedModel, finetuned_model_uri))

    # Handle adapter models separately
    if model_info.get("adapter"):
        for adapter_model in model_info["adapter"]:
            adapter_model_uri = KOAI[adapter_model.replace("-", "_").replace("/", "_")]
            g.add((adapter_model_uri, RDF.type, KOAI.AdaptedModel))
            g.add((adapter_model_uri, KOAI.isAdaptedFrom, model_uri))
            g.add((model_uri, KOAI.hasAdapterModel, adapter_model_uri))

    # Handle merged models separately
    if model_info.get("merge"):
        for merge_model in model_info["merge"]:
            merge_model_uri = KOAI[merge_model.replace("-", "_").replace("/", "_")]
            g.add((merge_model_uri, RDF.type, KOAI.MergedModel))
            g.add((merge_model_uri, KOAI.isMergedFrom, model_uri))
            g.add((model_uri, KOAI.hasMergedModel, merge_model_uri))

    # Handle quantized models separately
    if model_info.get("quantized"):
        for quantized_model in model_info["quantized"]:
            quantized_model_uri = KOAI[quantized_model.replace("-", "_").replace("/", "_")]
            g.add((quantized_model_uri, RDF.type, KOAI.QuantizedModel))
            g.add((quantized_model_uri, KOAI.isQuantizedFrom, model_uri))
            g.add((model_uri, KOAI.hasQuantizedModel, quantized_model_uri))

    # Handle datasets separately
    if model_info.get("dataset"):
        for dataset in model_info["dataset"]:
            dataset_uri = KOAI[dataset.replace("-", "_").replace("/", "_")]
            g.add((dataset_uri, RDF.type, KOAI.Dataset))
            g.add((dataset_uri, KOAI.isUsedBy, model_uri))
            g.add((dataset_uri, KOAI.name, rdflib.Literal(dataset)))
            g.add((model_uri, KOAI.usesDataset, dataset_uri))

In [25]:
# Specify the file path
output_file_path = "model_data.ttl"

# Load the existing RDF graph
g = Graph()
try:
    g.parse(output_file_path, format="turtle")
    print("Existing RDF graph loaded.")
except FileNotFoundError:
    print("No existing RDF file found. Creating a new one.")

# Add new model information
add_model_info_to_graph(data)

# Serialize and save the updated RDF graph
g.serialize(destination=output_file_path, format="turtle")

print(f"Updated RDF graph has been saved to {output_file_path}")

Existing RDF graph loaded.
Updated RDF graph has been saved to model_data.ttl
